# Algoritmo Apriori

##### Por: Antony isaac Huaman Hermoza

En este *notebook* se desarrollará el algoritmo Apriori para ser probada sobre una base de datos llamada `spotify.npy`. Posteriormente, se emplea el código de las reglas de asociación. Finalmente, se tiene una análisis segun el contenido propuesto en el archivo README.

## Librerías Utilizadas

- `numpy`: Para cargar los datos del spotify.npy.
- `collections`: Utilizamos su clase `defaultdict`, para los diccionarios.
- `pandas`: Para obtener el contador de las canciones.
- `altair`: Para obtener visualizaciones.

## Base de Datos

- `spotify.npy`: Es la base de datos que se utilizará, contiene listas de canciones.

**Importamos librerías**

In [1]:
import numpy as np
import collections
from collections import defaultdict
import pandas as pd
import itertools
import matplotlib.pyplot as plt  

## 1. Implementación del algoritmo

### Funciones Principales

- **get_frequent_itemsets(playlists, min_support)**

In [59]:
def get_frequent_itemsets(playlists, min_support):
  #Convertimos los datos en listas de conjuntos
  playlists = list(playlists.item().values())
  playlists = [set(playlist) for playlist in playlists]
  songs = [item for sublist in playlists for item in sublist]
  count_songs = pd.Series(data=songs).value_counts().to_dict()
  #Generamos diccionario de canciones y los indices de las playlists
  songs_playlists = collections.defaultdict(set)
  for index, playlist in enumerate(playlists):
      for cancion in playlist:
          songs_playlists[cancion].add(index)
  songs_playlists = songs_playlists
  #Obtenemos los itemsets de las canciones de acuerdo al umbral mínimo de soporte
  count = {cancion: veces for cancion, veces in count_songs.items() if veces / len(playlists) >= min_support}
  _itemset = [{cancion} for cancion in count.keys()]
  #Obtenemos los itemsets mas frecuentes
  _itemsets_frec = {}
  _itemsets_frec[1] = sorted(count.items(), key=lambda x: x[1], reverse=True)
  _itemsets_frec_size = []
  k = 2
  actual = _itemset   
  while len(actual) != 0:
    #Generaramos los itemsets candidatos
    combinaciones = set() 
    m = k - 2
    for candidato in actual: 
      candidato = list(candidato)
      for aux_candidato in actual:
        aux_candidato = list(aux_candidato)
        union = True
        for i in range(k - 2):
          if candidato[i] != aux_candidato[i]:
              union = False
              break
        if not union:
          continue
        if candidato[k - 2] < aux_candidato[k - 2]:
          c = candidato + [aux_candidato[k - 2]]
          c = frozenset(sorted(c))
          combinaciones.add(c)
    #Generar el itemset que cumplan con la condicion del umbral definido
    cont_combinaciones = {}   
    playlist_tamanio = len(playlists)   
    for candidato in combinaciones:
        playlists_inter = []
        for song in candidato:
          playlists_inter.append(songs_playlists[song])
        cont_combinaciones[candidato] = len(set.intersection(*playlists_inter))
    count_Aux = {subset: veces for subset, veces in cont_combinaciones.items() if veces / playlist_tamanio >= min_support} 
    diccionario_Aux = count_Aux.keys() 
    _itemsets_frec_size.extend(diccionario_Aux)
    _itemsets_frec[k] = sorted(count_Aux.items(), key=lambda x: x[1], reverse=True)
    k += 1
    actual = diccionario_Aux
  #Generaramos los itemsets en un dataframe
  _itemsets_df = pd.DataFrame([item for sublist in _itemsets_frec.values() for item in sublist]).round(3)
  _itemsets_df.columns = ["itemset", "contador_support"]
  _itemsets_df["support"] = _itemsets_df["contador_support"] / len(playlists)
  #Retornamos un vector con 4 datos
  extra_data = []
  #Primera parte con los itemsets mas frecuentes
  extra_data.append(_itemsets_df)
  #Segunda parte con los datos de la playlist para un posterior uso
  extra_data.append(playlists)
  #Tercera parte registro de canciones que estan en las playlists
  extra_data.append(songs_playlists)
  #Cuarta parte el registro de los itemsets largos 
  extra_data.append(_itemsets_frec_size)
  return extra_data

- **generate_association_rules(frequent_itemsets, confidence = 0, lift = 0)**

In [60]:
def generate_association_rules(frequent_itemsets, confidence, lift):
  reglas = []
  for itemset in frequent_itemsets[3]:
    #Se forman todas las combinaciones
    itemset_count = inter(itemset,frequent_itemsets[2])
    itemset_support = itemset_count / len(frequent_itemsets[0])
    for i in range(1, len(itemset) + 1):
        for x_set in combinations(itemset, i):
            x_set = set(x_set)
            y_set = set(itemset) - x_set
            x_support = inter(x_set,frequent_itemsets[2]) / len(frequent_itemsets[0])
            x_y_support = inter(x_set.union(y_set),frequent_itemsets[2]) / len(frequent_itemsets[0])
            rule_confidence = x_y_support / x_support
            if len(x_set) > 0 and len(y_set) > 0:
                y_support = inter(y_set,frequent_itemsets[2]) / len(frequent_itemsets[0])
                rule_lift = x_y_support / (x_support * y_support)
                reglas.append((x_set, y_set, rule_confidence, x_y_support, rule_lift))
  #Generar dataframe con las reglas de asociacion
  rule = pd.DataFrame(data=reglas, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  rule["X"] = list(map(tuple, rule["X"]))
  rule["Y"] = list(map(tuple, rule["Y"]))

  #Hallar las reglas de asociacion de 10 elementos para la confianza (Top 10)
  order_by="confidence"
  n=10
  confidence_df = rule.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion de 10 elementos para lift (Top 10)
  order_by="lift"
  n=10
  lift_df2 = rule.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion que cumplan confianza y lift
  rule["len_itemset"] = rule.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  rule=rule.sort_values(by="len_itemset", ascending=False)
  both_rules = rule[(rule["confidence"] >= confidence) & (rule["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules

### Funciones Secundarias de apoyo
- **inter(subset,songs_playlist)**

In [61]:
def inter(subset,songs_playlist):
    playlists_inter = []
    for song in subset:
        playlists_inter.append(songs_playlist[song])
    return len(set.intersection(*playlists_inter))

- **combinations(iterable, r)**

In [62]:
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        yield tuple(pool[i] for i in indices)

## 2. Aplicación del Algoritmo y reglas de asociación

**Lectura de datos**

In [63]:
database = np.load("spotify.npy", allow_pickle = True)

Para definir de mejor manera el mínimo soporte, analizamos las canciones más frecuentes del data set.

In [64]:
playlists = database.item().values()
unique_songs = [item for sublist in playlists for item in sublist]
pd.Series(unique_songs).value_counts().sort_values(ascending=False)[0:10]

Closer                         761
Home                           489
HUMBLE.                        470
Roses                          436
One Dance                      433
Ride                           429
Congratulations                410
Let Me Love You                403
Broccoli (feat. Lil Yachty)    402
Caroline                       395
dtype: int64

La canción que más veces aparece, lo hace tan solo 761, lo que equivale al 7%. Por esta razón, el mínimo soporte empleado será 0.01.

In [65]:
itemsets = get_frequent_itemsets(database,0.01)

Trabajaremos con los itemsets frecuentes con mayor soporte (top 10)

In [66]:
itemsets[0].sort_values(by="support").head()

,itemset,contador_support,support
1023,"(Mask Off, DNA., HUMBLE.)",100,0.01
999,"(Swang, iSpy (feat. Lil Yachty))",100,0.01
998,"(Congratulations, No Problem (feat. Lil Wayne ...",100,0.01
997,"(Bank Account, goosebumps)",100,0.01
996,"(Black Beatles, Fake Love)",100,0.01


**Filtramos las 10 mejores reglas de asociación**

Criterios de Calidad: **confidence** y **lift**

Generar reglas de asociación con confidence >= 0.5 y un lift >= 0.9

In [67]:
reglasConfidence,reglasLift,reglasAmbos = generate_association_rules(itemsets,0.5,0.9)

**Las reglas ordenadas con mayor confidencia(Top 10)**

In [68]:
reglasConfidence

,X,Y,confidence,support,lift
409,"(Mask Off, DNA.)","(HUMBLE.,)",0.909,0.098,2.002
416,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904
286,"(DNA.,)","(HUMBLE.,)",0.823,0.186,1.811
398,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.128,1.770
375,"(Broccoli (feat. Lil Yachty), Bounce Back)","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301
362,"(XO TOUR Llif3, Slippery (feat. Gucci Mane))","(HUMBLE.,)",0.765,0.099,1.685
446,"(Tunnel Vision, XO TOUR Llif3)","(HUMBLE.,)",0.750,0.100,1.652
385,"(Congratulations, Mask Off)","(HUMBLE.,)",0.747,0.118,1.645
440,"(Mask Off, Bounce Back)","(HUMBLE.,)",0.743,0.099,1.635
356,"(Mask Off, goosebumps)","(HUMBLE.,)",0.743,0.107,1.637


**Podemos obtener las reglas con mayor lift(Top 10)**

In [69]:
reglasLift

,X,Y,confidence,support,lift
65,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
64,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
293,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
292,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261
199,"(You Was Right,)","(Money Longer,)",0.603,0.114,3.183
198,"(Money Longer,)","(You Was Right,)",0.603,0.114,3.183
308,"(Bank Account,)","(Butterfly Effect,)",0.483,0.108,2.547
309,"(Butterfly Effect,)","(Bank Account,)",0.572,0.108,2.547
195,"(Magnolia,)","(Bank Account,)",0.571,0.110,2.541
194,"(Bank Account,)","(Magnolia,)",0.491,0.110,2.541


**Reglas que cumplen con el límite mayor a los umbrales de confidencia y lift al mismo tiempo(top 10)**

In [70]:
reglasAmbos.head(10)

,X,Y,confidence,support,lift,len_itemset
447,"(XO TOUR Llif3, HUMBLE.)","(Tunnel Vision,)",0.500,0.100,2.073,3
362,"(XO TOUR Llif3, Slippery (feat. Gucci Mane))","(HUMBLE.,)",0.765,0.099,1.685,3
367,"(Congratulations, HUMBLE.)","(goosebumps,)",0.523,0.109,1.740,3
368,"(Congratulations, goosebumps)","(HUMBLE.,)",0.723,0.109,1.591,3
369,"(HUMBLE., goosebumps)","(Congratulations,)",0.671,0.109,1.704,3
373,"(Bad and Boujee (feat. Lil Uzi Vert), Broccoli...","(Bounce Back,)",0.645,0.098,2.224,3
374,"(Bad and Boujee (feat. Lil Uzi Vert), Bounce B...","(Broccoli (feat. Lil Yachty),)",0.592,0.098,1.519,3
375,"(Broccoli (feat. Lil Yachty), Bounce Back)","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301,3
379,"(XO TOUR Llif3, HUMBLE.)","(goosebumps,)",0.544,0.108,1.809,3
380,"(XO TOUR Llif3, goosebumps)","(HUMBLE.,)",0.730,0.108,1.608,3


## Explicación de las reglas obtenidas

In [71]:
Reglas_Top_4=reglas_lift.head(4)

In [72]:
Reglas_Top_4

,X,Y,confidence,support,lift
65,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
64,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
293,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
292,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261


### Regla 1

- (No Heart,) ---> (X (feat. Future),)

Ambas canciones son de genero rap y puede existir relacion. El valor de la confianza de la regla alcanza 0.505 y supera ligeramente el umbral definido (0.5), se puede decir que la regla se cumple según ese criterio.

### Regla 2


- (X (feat. Future),) ---> (No Heart,)

Podemos ver las canciones de la anterior regla, esto nos confirma que estas canciones estan ligadas al género de música. También podemos notar que la confianza supera el umbral predefinido y cumple con el criterio.



### Regla 3

- (Chicken Fried,) ---> (Knee Deep (feat. Jimmy Buffett),)

Podemos notar que ambas canciones son de la misma banda, por lo cual su relacion esta ligada por el gusto musical a la banda. El umbral de confianza de aparicion de esta regla se iguala con la definida y la cumple.

### Regla 4

- (Knee Deep (feat. Jimmy Buffett),) ---> (Chicken Fried,)

Podemos encontrar las mismas canciones de acuerdo a la regla anterior, por ello podemos decir concluir que existe una fuerte relación. Ahora la confianza alcanza 0.682 y supera el umbral definido, podemo concluir que la regla ofrece mayor fidelidad gracias a este criterio.


## 4. Analisis de frecuencia de canciones
Mostraremos las 30 canciones mas frecuentes en todas las playlists.

In [58]:
playlists = database.item().values()
Songs = [item for sublist in playlists for item in sublist]
pd.Series(Songs).value_counts().sort_values(ascending=False)[0:30]

Closer                                     761
Home                                       489
HUMBLE.                                    470
Roses                                      436
One Dance                                  433
Ride                                       429
Congratulations                            410
Let Me Love You                            403
Broccoli (feat. Lil Yachty)                402
Caroline                                   395
Gold                                       386
Stay                                       355
T-Shirt                                    355
Bad and Boujee (feat. Lil Uzi Vert)        353
Riptide                                    339
iSpy (feat. Lil Yachty)                    339
Forever                                    337
Unforgettable                              331
XO TOUR Llif3                              330
Sorry                                      330
Location                                   326
Slide        